In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ngsolve  # noqa: F401
else:
    try:
        import ngsolve
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/ngsolve-install.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        import ngsolve  # noqa: F401

In [ ]:
# Adapted from ngsxfem-jupyter/cutfem.ipynb

In [ ]:
from ngsolve import *
from xfem import *
from netgen.geom2d import SplineGeometry

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5, -1.5], [1.5, 1.5], bc=1)
mesh = Mesh(square.GenerateMesh(maxh=0.4, quad_dominated=False))
levelset = (sqrt(x * x + y * y) - 1.0)
lsetp1 = GridFunction(H1(mesh, order=1))
InterpolateToP1(levelset, lsetp1)

In [ ]:
Vh = H1(mesh, order=2, dirichlet=[1, 2, 3, 4])
ci = CutInfo(mesh, lsetp1)
VhG = FESpace([Compress(Vh, GetDofsOfElements(Vh, ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS]])
freedofs = VhG.FreeDofs()
gfu = GridFunction(VhG)
gfu.components[0].Set(1)
gfu.components[1].Set(-1)
print(Vh.ndof, VhG.components[0].ndof, VhG.components[1].ndof)

In [ ]:
kappaminus = CutRatioGF(ci)
kappa = (kappaminus, 1 - kappaminus)
n = Normalize(grad(lsetp1))
h = specialcf.mesh_size
alpha = [1.0, 20.0]
stab = 20 * (alpha[1] + alpha[0]) / h

In [ ]:
u, v = VhG.TnT()
average_flux_u = sum([- kappa[i] * alpha[i] * grad(u[i]) * n for i in [0, 1]])
average_flux_v = sum([- kappa[i] * alpha[i] * grad(v[i]) * n for i in [0, 1]])
jump_u = u[0] - u[1]
jump_v = v[0] - v[1]

In [ ]:
dx_neg = dCut(levelset=lsetp1, domain_type=NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type=POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type=IF, definedonelements=ci.GetElementsOfType(IF))

In [ ]:
a = BilinearForm(VhG, symmetric=True)
a += alpha[0] * grad(u[0]) * grad(v[0]) * dx_neg
a += alpha[1] * grad(u[1]) * grad(v[1]) * dx_pos
a += (average_flux_u * jump_v + average_flux_v * jump_u + stab * jump_u * jump_v) * ds
a.Assemble()

In [ ]:
coef_f = [1, 0]
f = LinearForm(VhG)
f += coef_f[0] * v[0] * dx_neg
f += coef_f[1] * v[1] * dx_pos
f.Assemble()

In [ ]:
gfu.vec[:] = 0
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs) * f.vec